In [12]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [ ]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [3]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmenge

In [4]:
H = [n for n in range(1, 25)]

## Batterie-Systemspezifikationen

In [5]:
wirkungsgrad_wechselrichter = 0.985
wirkungsgrad_laden = 0.975
round_trip_efficiency = 0.95

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-((wirkungsgrad_laden-round_trip_efficiency) / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter
wirkungsgrad_systemausgang = (1-((wirkungsgrad_laden-round_trip_efficiency) / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter
wirkungsgrad_systemausgang = (((wirkungsgrad_laden) / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter


f_e = wirkungsgrad_systemeingang # Faktor Eingang
f_v = wirkungsgrad_systemausgang # Faktor Ausgang


nennkapazität = 40 # MWh brutto
lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbesand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
zykluskosten = 1500 # € / nettokapazität
mwh_zykluskosten = zykluskosten / nettokapazität # € / MWh

erlaubte_zyklen_pro_tag = 2

a = anfangsbesand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität 

c = 0.5 # nennkapazität / h

print(wirkungsgrad_systemeingang)
print(wirkungsgrad_systemausgang)

print(wirkungsgrad_systemeingang*wirkungsgrad_systemausgang)
print(wirkungsgrad_wechselrichter*wirkungsgrad_wechselrichter*round_trip_efficiency)


0.960375
0.9597435897435898
0.92171375
0.92171375


## Vorhersagedaten

In [6]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

p_h = {}

for h in H:
    stundenprognose = prognose[prognose['Stunde'] == h]
    checksum = stundenprognose['Wahrscheinlichkeit'].sum()
    if checksum != 1:
        print('WARNING: Prognosedaten unvollständig')

    erwartungswert_strompreis = (stundenprognose['Strompreis'] * stundenprognose['Wahrscheinlichkeit']).sum() / checksum
    p_h[h] = erwartungswert_strompreis

print (p_h)

{1: np.float64(81.52000000000001), 2: np.float64(72.24000000000001), 3: np.float64(68.16000000000001), 4: np.float64(66.64), 5: np.float64(66.8), 6: np.float64(71.04), 7: np.float64(88.55999999999999), 8: np.float64(92.68), 9: np.float64(80.92), 10: np.float64(61.080000000000005), 11: np.float64(43.080000000000005), 12: np.float64(30.880000000000003), 13: np.float64(22.44), 14: np.float64(15.959999999999999), 15: np.float64(12.52), 16: np.float64(18.44), 17: np.float64(34.88), 18: np.float64(56.56), 19: np.float64(78.44000000000001), 20: np.float64(105.60000000000001), 21: np.float64(144.8), 22: np.float64(125.12), 23: np.float64(101.36), 24: np.float64(85.28)}


## Entscheidungsvariablen

In [7]:
e_h={}
v_h={}

for h in H:
    e_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}")
    v_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}")

print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [e_1, v_1, e_2, v_2, e_3, v_3, e_4, v_4, e_5, v_5, e_6, v_6, e_7, v_7, e_8, v_8, e_9, v_9, e_10, v_10, e_11, v_11, e_12, v_12, e_13, v_13, e_14, v_14, e_15, v_15, e_16, v_16, e_17, v_17, e_18, v_18, e_19, v_19, e_20, v_20, e_21, v_21, e_22, v_22, e_23, v_23, e_24, v_24]


## Zielfunktion

In [8]:
gewinn_kauf_verkauf = quicksum((p_h[h] * (v_h[h] - e_h[h])) for h in H)
zykluskosten = quicksum((e_h[h] * f_e * mwh_zykluskosten) for h in H)
zykluskosten = 3000

scip.setObjective(gewinn_kauf_verkauf - zykluskosten, sense="maximize")

## Nebenbedingungen / Restriktionen

In [9]:
# Ladestand zur Stunde 0 = Ladestand zur Stunde 24
scip.addCons(quicksum(((e_h[h] * f_e) - (v_h[h] / f_v)) for h in H) == 0, name="Anfangs- und Endbestand gleich") # Check

# Maximale Ladezyklen am pro Tag
scip.addCons(quicksum((e_h[h] * f_e) for h in H) <= (2 * nettokapazität), name="Maximale Ladezyklen pro Tag") # Check

# Mindestladestand nicht unterschritten
for t in H:
    H_t =  [n for n in range(1, t+1)]
    scip.addCons( (a + quicksum(((e_h[h] * f_e) - (v_h[h] / f_v)) for h in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t{t}")
    scip.addCons( (a + quicksum(((e_h[h] * f_e) - (v_h[h] / f_v)) for h in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t{t}")

for h in H:
    scip.addCons((e_h[h] * f_e) + (v_h[h] / f_v) <= c * nennkapazität)

print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Anfangs- und Endbestand gleich, Maximale Ladezyklen pro Tag, Mindestladestand zum Zeitpunkt t1, Maximalladestand zum Zeitpunkt t1, Mindestladestand zum Zeitpunkt t2, Maximalladestand zum Zeitpunkt t2, Mindestladestand zum Zeitpunkt t3, Maximalladestand zum Zeitpunkt t3, Mindestladestand zum Zeitpunkt t4, Maximalladestand zum Zeitpunkt t4, Mindestladestand zum Zeitpunkt t5, Maximalladestand zum Zeitpunkt t5, Mindestladestand zum Zeitpunkt t6, Maximalladestand zum Zeitpunkt t6, Mindestladestand zum Zeitpunkt t7, Maximalladestand zum Zeitpunkt t7, Mindestladestand zum Zeitpunkt t8, Maximalladestand zum Zeitpunkt t8, Mindestladestand zum Zeitpunkt t9, Maximalladestand zum Zeitpunkt t9, Mindestladestand zum Zeitpunkt t10, Maximalladestand zum Zeitpunkt t10, Mindestladestand zum Zeitpunkt t11, Maximalladestand zum Zeitpunkt t11, Mindestladestand zum Zeitpunkt t12, Maximalladestand zum Zeitpunkt t12, Mindestladestand zum Zeitpunkt t13, Maximalladestand zum Zeitpunkt t13, M

## Berechnung der Lösung

In [10]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
        print("EINKAUF Stunde", h, " : " , scip.getVal(e_h[h]))
        print("Verkauf Stunde", h, " : " , scip.getVal(v_h[h]))

else:
    print('Problem hat keine Lösung')



Status des Solvers: optimal 
LP Solver <Soplex 7.1.1>: barrier convergence tolerance cannot be set -- tolerance of SCIP and LP solver may differ
LP Solver <Soplex 7.1.1>: fastmip setting not available -- SCIP parameter has no effect
LP Solver <Soplex 7.1.1>: number of threads settings not available -- SCIP parameter has no effect
transformed problem has 48 variables (0 bin, 0 int, 0 impl, 48 cont) and 74 constraints
     74 constraints of type <linear>

original problem has 1320 active (37.1622%) nonzeros and 1320 (37.1622%) check nonzeros

feasible solution found by trivial heuristic after 0.0 seconds, objective value -3.000000e+03
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 96 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 1 del conss, 0 add conss, 96 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 0 del vars, 25 del conss, 0 add conss, 96 chg bounds, 0 chg s

In [11]:
einkauf_sum = sum(scip.getVal(e_h[h]) for h in H)
print(f'Es wurden {einkauf_sum} MWh eingekauft')
verkauf_sum = sum(scip.getVal(v_h[h]) for h in H)
print(f'Es wurden {verkauf_sum} MWh verkauft')

gesamtwirkungsgrad = (0.95*0.985*0.985)


if(gesamtwirkungsgrad != verkauf_sum / einkauf_sum):
    print('Wirkungsgrad stimmt nicht überein')
else:
    print('Wirkungsgrad stimmt überein')

Es wurden 66.64063516855396 MWh eingekauft
Es wurden 61.42358974358975 MWh verkauft
Wirkungsgrad stimmt überein
